In [1]:

from dask.distributed import Client
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

In [2]:
client = Client() 

2025-11-16 10:46:33,327 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "C:\Users\ultra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tornado\websocket.py", line 965, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ultra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tornado\web.py", line 3375, in wrapper
    return method(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ultra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\loca

In [10]:
dt_file = './../data/test-parsed.csv'
df_file = './../data/train-parsed.csv'

dt = dd.read_csv(dt_file, sep="," ) # Archiu de test processat
df = dd.read_csv(df_file, sep="," ) # Arxiu de train processat

dt = dt.drop(columns=["ID"]) # Traiem la columna ID del test
df = df.dropna()
df.head()

,num_week_iso,num_stores,num_sizes,price,year,weekly_demand,week_iso,R,G,B,...,f502,f503,f504,f505,f506,f507,f508,f509,f510,f511
0,1,152,5,35.99,2023,69,1,255,215,0,...,0.105586,-0.31216,0.35252,-0.241651,0.464914,0.169642,-0.314308,-0.158037,-0.022556,-0.080948
1,2,152,5,35.99,2023,112,2,255,215,0,...,0.105586,-0.31216,0.35252,-0.241651,0.464914,0.169642,-0.314308,-0.158037,-0.022556,-0.080948
2,3,152,5,35.99,2023,135,3,255,215,0,...,0.105586,-0.31216,0.35252,-0.241651,0.464914,0.169642,-0.314308,-0.158037,-0.022556,-0.080948
3,4,152,5,35.99,2023,99,4,255,215,0,...,0.105586,-0.31216,0.35252,-0.241651,0.464914,0.169642,-0.314308,-0.158037,-0.022556,-0.080948
4,5,152,5,35.99,2023,74,5,255,215,0,...,0.105586,-0.31216,0.35252,-0.241651,0.464914,0.169642,-0.314308,-0.158037,-0.022556,-0.080948


In [5]:
# AIXO NOMES CAL PER QUAN PROVEM AMB TEST.CSV
dt = dt[df.columns] # Ens assegurem que les columnes del test siguin les mateixes que les del train, i estiguin en el mateix ordre
dt.head()

,num_week_iso,num_stores,num_sizes,price,year,weekly_demand,week_iso,R,G,B,...,f502,f503,f504,f505,f506,f507,f508,f509,f510,f511
0,1,118,5,29.99,2025,0,7,245,244,238,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
1,2,118,5,29.99,2025,0,8,245,244,238,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
2,3,118,5,29.99,2025,0,9,245,244,238,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
3,4,118,5,29.99,2025,0,10,245,244,238,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
4,5,118,5,29.99,2025,0,11,245,244,238,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877


In [6]:
#Comproebem els tipus de dades (només int64 i float64)
df.dtypes

num_week_iso      int64
num_stores        int64
num_sizes         int64
price           float64
year              int64
                 ...   
f507            float64
f508            float64
f509            float64
f510            float64
f511            float64
Length: 594, dtype: object

In [8]:
train=False

if train:
    X = df.drop(columns=["weekly_demand"]) # Variables independents, totes excepte la que volem predir
    y = df[ ['weekly_demand']] # Variable que volem predir
else: # Si no entrenem, preparem les dades de test
    X = dt.drop(columns=["weekly_demand"]) # Variables independents, totes excepte la que volem predir i l'ID
    # la y no cal pq es la que volem predir


X.head()

,num_week_iso,num_stores,num_sizes,price,year,week_iso,R,G,B,"category_Accesories, Swim and Intimate",...,f502,f503,f504,f505,f506,f507,f508,f509,f510,f511
0,1,118,5,29.99,2025,7,245,244,238,0,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
1,2,118,5,29.99,2025,8,245,244,238,0,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
2,3,118,5,29.99,2025,9,245,244,238,0,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
3,4,118,5,29.99,2025,10,245,244,238,0,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877
4,5,118,5,29.99,2025,11,245,244,238,0,...,0.324355,-0.80481,0.347086,-0.7558,0.816484,0.648224,-0.254482,0.305508,0.196883,-0.016877


In [9]:
X_np = X.compute()
if train:
    y_np = y.compute()#.ravel()

In [ ]:
#AIXO NOMES CAL EXECUTAR-LO SI ESTEM ENTRENANT
if train:
    model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
    model.fit(X_np, y_np)

C:\Users\ultra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
# Per guardar el model entrenat
import os
import joblib

os.makedirs("./../models", exist_ok=True)
model_path = "./../models/rf_model_v2.joblib"
joblib.dump(model, model_path, compress=3) 
print("Modelo guardado en:", model_path)

Modelo guardado en: ./models/rf_model_v2.joblib


In [11]:
# Per carregar el model entrenat
import joblib
model = joblib.load("./../models/rf_model_v2.joblib")
# usar model_loaded.predict(...)

In [12]:
# Predicció
y_pred = model.predict(X_np)



In [13]:
# Aqui creem una copia del CSV original, pero amb les dades predites


import numpy as np
import pandas as pd


y_pred_full = y_pred.ravel()
y_pred_int = np.rint(y_pred_full).astype(int)
y_pred_int = np.clip(y_pred_int, 0, None)

# llegir CSV original
df_orig = pd.read_csv(dt_file if not train else df_file)
assert len(df_orig) == len(y_pred_int), "Mismatch entre filas originales y predicciones"

# afegir columna amb prediccions
df_orig['weekly_demand'] = y_pred_int

# guardar nuevo CSV
out_path = './../data/test-parsed-with-predictions-rf.csv' if not train else './../data/train-parsed-with-predictions-rf.csv'
df_orig.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: ./../data/test-parsed-with-predictions-rf.csv


In [ ]:

# Aixo serveix per avaluar les prediccions (si estem en mode train)
if train:
    df_p = df_orig.copy()
    df_p = df_p[["ID",'weekly_demand']]

    df_p["predicted_weekly_demand"] =  1*y_pred_int
    df_s = df_p.groupby('ID', as_index=False)['weekly_demand'].sum()
    df_s["predicted_weekly_demand"] = df_p.groupby('ID', as_index=False)['predicted_weekly_demand'].sum()["predicted_weekly_demand"]

    df_s["lost_sales"] = np.maximum(0, df_s["weekly_demand"] - df_s["predicted_weekly_demand"])
    df_s["excess stock"] = np.maximum(0, df_s["predicted_weekly_demand"] - df_s["weekly_demand"])

    average_lost_sales = df_s["lost_sales"].mean()
    average_excess_stock = df_s["excess stock"].mean()
    pd.set_option('display.max_rows', 20)
    df_s.head(200)
    #print("Average lost sales:", average_lost_sales)
    #print("Average excess stock:", average_excess_stock) 


,ID,weekly_demand,predicted_weekly_demand,lost_sales,excess stock
0,1,806,834,0,28
1,2,2266,2309,0,43
2,3,63791,63999,0,208
3,4,11004,11368,0,364
4,6,14684,14984,0,300
...,...,...,...,...,...
195,238,2219,2214,5,0
196,239,8844,9035,0,191
197,241,10713,10961,0,248
198,242,188,204,0,16
